In [ ]:
import torch
import torch.nn
import torch.optim
import torch.utils.data
import torch.nn.functional as F
from splitcross import SplitCrossEntropyLoss

import numpy as np
import networkx as nx
import math
import json
import time

### Imported from nasbench nlp
import data
import os
from utils import batchify
from argparse import Namespace
from model import AWDRNNModel
from train import train, evaluate
import datetime

In [ ]:
%pwd

In [3]:
%cd machop

/home/ansonhon/mase_project/machop


In [4]:
from naslib.search_spaces.nasbenchnlp.graph import NasBenchNLPSearchSpace
from naslib.utils.get_dataset_api import get_nlp_api


In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader

In [7]:
graph = NasBenchNLPSearchSpace()

In [15]:
graph.sample_random_architecture(None)
new_compact = graph.get_compact()

In [ ]:
dir(graph)

In [23]:
def make_compact_mutable(compact):
    # convert tuple to list so that it is mutable
    edge_list = []
    for edge in compact[0]:
        edge_list.append(list(edge))
    return [edge_list, list(compact[1]), list(compact[2])]

In [16]:
compact = new_compact
compact = make_compact_mutable(compact)
edges, ops, hiddens = compact

nbr = NasBenchNLPSearchSpace()
nbr.set_compact(new_compact)
nbr_model = torch.nn.Module()
nbr_model.arch = nbr


idx_choices = [i for i in range(len(ops)) if ops[i] not in [0, 6, 7]]
for idx in idx_choices:
    num_inputs = len([edge for edge in edges if edge[1] == idx])
    groups = [[0], [1, 2, 3], [4, 5]]
    group = groups[num_inputs]
    choices = [i for i in group if i != ops[idx]]
    for choice in choices:
        new_ops = ops.copy()
        new_ops[idx] = choice


In [24]:
import copy
"""
Return all neighbors of the architecture
Currently has the same todo's as in mutate()
"""
compact = new_compact
compact = make_compact_mutable(compact)
edges, ops, hiddens = compact
nbhd = []

def add_to_nbhd(new_compact, nbhd):
    nbr = NasBenchNLPSearchSpace()
    nbr.set_compact(new_compact)
    nbr_model = torch.nn.Module()
    nbr_model.arch = nbr
    nbhd.append(nbr_model)
    return nbhd

# add op neighbors
idx_choices = [i for i in range(len(ops)) if ops[i] not in [0, 6, 7]]
for idx in idx_choices:
    num_inputs = len([edge for edge in edges if edge[1] == idx])
    groups = [[0], [1, 2, 3], [4, 5]]
    group = groups[num_inputs]
    choices = [i for i in group if i != ops[idx]]
    for choice in choices:
        new_ops = ops.copy()
        new_ops[idx] = choice
        nbhd = add_to_nbhd([copy.deepcopy(edges), new_ops, hiddens.copy()], nbhd)

# add edge neighbors
edge_choices = [i for i in range(len(edges)) if edges[i][0] >= 4]
for i in edge_choices:
    node_choices = [j for j in range(4, edges[i][1])]
    for j in node_choices:
        new_edges = copy.deepcopy(edges)
        new_edges[i][0] = j
        nbhd = add_to_nbhd([new_edges, ops.copy(), hiddens.copy()], nbhd)



In [29]:
nbr_model.state_dict()

OrderedDict()

In [ ]:
dir(nbr_model.state_dict())

In [1]:
### Prepare dataloader for obtaining proxy values
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

data = np.load(r'/home/ansonhon/mase_project/machop/nas-bench-nlp-release/data/ptb/ptb_train_data_tokenized.npy')

text_data_tensor = torch.tensor(data)
dataset = TensorDataset(text_data_tensor)
batch_size = 20
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [32]:
def convert_compact_to_recipe(compact):
    nodes = ['x', 'h_prev_0', 'h_prev_1', 'h_prev_2']
    op_dict = ['in', 'activation_sigm', 'activation_tanh', 'activation_leaky_relu',
               'elementwise_sum', 'elementwise_prod', 'linear', 'blend']

    edges, ops, hiddens = compact
    max_node_idx = max([max(edge) for edge in edges])

    # create the set of node names
    reg_node_idx = 0
    hidden_node_idx = 0
    for i in range(len(nodes), max_node_idx + 1):
        if i not in hiddens:
            nodes.append('node_{}'.format(reg_node_idx))
            reg_node_idx += 1
        else:
            nodes.append('h_new_{}'.format(hidden_node_idx))
            hidden_node_idx += 1

    recipe = {}
    for i in range(4, len(nodes)):
        node_dict = {}
        node_dict['op'] = op_dict[ops[i]]
        inputs = []
        for edge in edges:
            if edge[1] == i:
                inputs.append(nodes[edge[0]])
        node_dict['input'] = inputs
        recipe[nodes[i]] = node_dict

    return recipe

In [ ]:
test_receipe = convert_compact_to_recipe(new_compact)
test_receipe

In [42]:
tuple_list = []

while len(tuple_list) < 10:
    temp = graph.sample_random_architecture(None)
    if temp not in tuple_list:
        graph



        tuple_list.append(temp)


In [27]:
['epe_nas', 'fisher', 'grad_norm', 'grasp', 'jacov', 'l2_norm', 'nwot', 'plain', 'snip', 'synflow', 'zen', 'params', 'flops']
from naslib.predictors.utils.pruners.predictive import find_measures

metrics = {}
dataloader=self.data_module.train_dataloader()
dataload_info=["random",len(dataloader),10]
loss_function = nn.MSELoss()
model=model.model
device = torch.device('cuda')
model.to(device)
num_batches_to_keep = 1

In [ ]:
from naslib.predictors import ZeroCost
from naslib.predictors.utils.pruners.predictive import find_measures
zc_proxy_list = ['epe_nas', 'fisher', 'grad_norm', 'grasp', 'jacov', 'l2_norm', 'nwot', 'plain', 'snip', 'synflow', 'zen', 'params', 'flops']
zc_proxy = zc_proxy_list[0]

zc_predictor = ZeroCost(method_type=zc_proxy)
score = zc_predictor.query(graph=graph, dataloader= train_loader)
print(score)